In [4]:
import os
from typing import Mapping

import torch as t
from beartype import beartype as typed
from datasets import load_dataset
from language_modeling import explore_batch
from tokenization import dependencies_tokenizer
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments


In [ ]:
repo_name = open("SECRET.txt").read().strip()
dataset = load_dataset(f"{repo_name}/flat_shuffle")
model_name = "roneneldan/TinyStories-8M"
tokenizer = dependencies_tokenizer(vocab_size=500)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
model.resize_token_embeddings(len(tokenizer))
for layer in model.parameters():
    layer.data = t.randn_like(layer.data) * 0.01


In [ ]:
tokens_sample = tokenizer(dataset["train"][0]["text"])["input_ids"]
print(len(tokens_sample))
print(tokens_sample[:10])


In [ ]:
@typed
def tokenize_function(example: Mapping[str, str | int]) -> Mapping[str, list[int]]:
    result = tokenizer(example["text"])
    result["labels"] = result["input_ids"]
    return result


subset_size = 60000
subset = dataset["train"].select(range(subset_size)).to_iterable_dataset()
tokenized = subset.map(tokenize_function, batched=True).remove_columns(["text"])


In [ ]:
explore_batch(model, tokenizer, tokenized)


In [ ]:
batch_size = 8

training_args = TrainingArguments(
    output_dir="trainer",
    per_device_train_batch_size=batch_size,
    learning_rate=1e-3,
    logging_steps=50,
    num_train_epochs=1,
    max_steps=subset_size // batch_size,
    save_total_limit=1,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
)
trainer.train()


In [ ]:
explore_batch(model, tokenizer, tokenized)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

name = input("Model name:")
model.push_to_hub(name)
tokenizer.push_to_hub(name)
